# 📦 Import Modules

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# 🗃️ Load preprocessed data

In [2]:
X_train = np.load('../data/processed/X_train.npy')
X_test = np.load('../data/processed/X_test.npy')
y_train = np.load('../data/processed/y_train.npy')
y_test = np.load('../data/processed/y_test.npy')

# 🔄 Convert to tensors

In [3]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [4]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 🧠 Define model

In [5]:
class FraudNet(nn.Module):
    def __init__(self, input_dim):
        super(FraudNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

In [6]:
model = FraudNet(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🚀 Training loop

In [7]:
epochs = 10
for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.5946
Epoch [2/10], Loss: 0.5298
Epoch [3/10], Loss: 0.4813
Epoch [4/10], Loss: 0.4073
Epoch [5/10], Loss: 0.2814
Epoch [6/10], Loss: 0.3541
Epoch [7/10], Loss: 0.4065
Epoch [8/10], Loss: 0.3352
Epoch [9/10], Loss: 0.3429
Epoch [10/10], Loss: 0.2735


# 📊 Evaluation

In [8]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy()
    y_pred_class = (y_pred > 0.5).astype(int)

In [9]:
print("\n✅ Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_class))


✅ Confusion Matrix:
[[145   5]
 [ 14 132]]


In [10]:
print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred_class, digits=4))


✅ Classification Report:
              precision    recall  f1-score   support

           0     0.9119    0.9667    0.9385       150
           1     0.9635    0.9041    0.9329       146

    accuracy                         0.9358       296
   macro avg     0.9377    0.9354    0.9357       296
weighted avg     0.9374    0.9358    0.9357       296



In [11]:
print("\n✅ ROC AUC Score:")
print(roc_auc_score(y_test, y_pred))


✅ ROC AUC Score:
0.9525570776255707


# 💾 Save trained model

In [12]:
torch.save(model.state_dict(), '../app/models/fraud_model.pth')
print("✅ Model saved as fraud_model.pth")

✅ Model saved as fraud_model.pth
